In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.stats import f_oneway
from scipy.stats import chi2_contingency

In [2]:
!pip install kmodes

In [3]:
pd.set_option('display.max_columns', None)

# Preparing dataframe to be used

In [19]:
df = pd.read_csv("FLIGHTS.csv")
df.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,MKT_CARRIER_AIRLINE_ID,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_NAME,CRS_DEP_TIME,DEP_TIME,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,ARR_TIME,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,CANCELLED,CANCELLATION_CODE,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,index,CRS_DEP_DT,lagged_delay_flag,lag_delay_minutes,prev_real_delay
0,2024,1,1,1,1,19393,10140,1014005,"Albuquerque, NM",10423,1042302,"Austin, TX",700,707.0,7.0,0.0,0.0,950.0,15.0,1.0,1.0,0.0,NaN,95.0,103.0,619.0,7.0,0.0,8.0,0.0,0.0,0,2024-01-01 07:00:00,0,0.0,0.0
1,2024,1,1,1,1,19393,10140,1014005,"Albuquerque, NM",10423,1042302,"Austin, TX",1830,1826.0,0.0,0.0,-1.0,2112.0,2.0,0.0,0.0,0.0,NaN,100.0,106.0,619.0,0.0,0.0,0.0,0.0,0.0,1,2024-01-01 18:30:00,0,0.0,0.0
2,2024,1,1,1,1,19393,10140,1014005,"Albuquerque, NM",10800,1080003,"Burbank, CA",1420,1426.0,6.0,0.0,0.0,1516.0,0.0,0.0,-1.0,0.0,NaN,130.0,110.0,672.0,0.0,0.0,0.0,0.0,0.0,2,2024-01-01 14:20:00,0,0.0,0.0
3,2024,1,1,1,1,19393,10140,1014005,"Albuquerque, NM",10821,1082106,"Baltimore, MD",1500,1514.0,14.0,0.0,0.0,2050.0,15.0,1.0,1.0,0.0,NaN,215.0,216.0,1670.0,14.0,0.0,1.0,0.0,0.0,3,2024-01-01 15:00:00,0,0.0,0.0
4,2024,1,1,1,1,19393,10140,1014005,"Albuquerque, NM",11259,1125904,"Dallas, TX",530,527.0,0.0,0.0,-1.0,805.0,0.0,0.0,-1.0,0.0,NaN,105.0,98.0,580.0,0.0,0.0,0.0,0.0,0.0,4,2024-01-01 05:30:00,0,0.0,0.0


In [20]:
df.drop(columns=["lag_delay_minutes", "index", "CRS_DEP_DT"], inplace=True)

In [21]:
# This is Chloe's code for feature engineering

# ---------------------------------------------------------
# Create FL_DATE (needed for daily counts)
# ---------------------------------------------------------
df = df.rename(columns={'DAY_OF_MONTH': 'DAY'}) # for some reason was complaining about day_of_month
df['FL_DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']])

# ---------------------------------------------------------
# 1) ORIGIN BUCKET (quartiles of flights per day per origin)
# ---------------------------------------------------------
origin_counts = (
    df.groupby(['FL_DATE', 'ORIGIN_AIRPORT_ID'])
      .size()
      .reset_index(name='origin_flights_day')
)

df = df.merge(origin_counts, on=['FL_DATE', 'ORIGIN_AIRPORT_ID'], how='left')

df['origin_bucket'] = pd.qcut(df['origin_flights_day'], q=4, labels=[1, 2, 3, 4]) #bottom quartile is 1, top quartile is 4

# ---------------------------------------------------------
# 2) DESTINATION BUCKET (quartiles of flights per day per destination)
# ---------------------------------------------------------
dest_counts = (
    df.groupby(['FL_DATE', 'DEST_AIRPORT_ID'])
      .size()
      .reset_index(name='dest_flights_day')
)

df = df.merge(dest_counts, on=['FL_DATE', 'DEST_AIRPORT_ID'], how='left')

df['destination_bucket'] = pd.qcut(df['dest_flights_day'], q=4, labels=[1, 2, 3, 4]) #bottom quartile is 1, top quartile is 4

# ---------------------------------------------------------
# 3) DISTANCE BUCKET (quartiles of distance)
# ---------------------------------------------------------
df['distance_bucket'] = pd.qcut(df['DISTANCE'], q=4, labels=[1, 2, 3, 4]) #bottom quartile is 1, top quartile is 4

# ---------------------------------------------------------
# 4) AIRLINE BUCKET
# bottom 6 = bucket 0
# top 4    = bucket 1
# ---------------------------------------------------------
airline_counts = df['MKT_CARRIER_AIRLINE_ID'].value_counts()

bottom_6 = airline_counts.sort_values().index[:6]
top_4 = airline_counts.sort_values(ascending=False).index[:4]

df['airline_bucket'] = None
df.loc[df['MKT_CARRIER_AIRLINE_ID'].isin(bottom_6), 'airline_bucket'] = 0
df.loc[df['MKT_CARRIER_AIRLINE_ID'].isin(top_4), 'airline_bucket'] = 1

# ---------------------------------------------------------
# Final preview
# ---------------------------------------------------------
df[['origin_bucket', 'destination_bucket', 'distance_bucket', 'airline_bucket']].head()

,origin_bucket,destination_bucket,distance_bucket,airline_bucket
0,1,2,2,1
1,1,2,2,1
2,1,1,3,1
3,1,2,4,1
4,1,2,2,1


In [22]:
df_completed = df[df['CANCELLED'] == 0] # Use non cancelled flights + base off of departure delay not arrival delay
df_completed.shape

(7444080, 40)

In [23]:
df_completed['HOUR'] = df_completed['CRS_DEP_TIME'] // 100 
df_completed.head()

/var/folders/h1/w_jtl0654ns96y8ngd075fj40000gn/T/ipykernel_7650/1921052717.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_completed['HOUR'] = df_completed['CRS_DEP_TIME'] // 100


,YEAR,QUARTER,MONTH,DAY,DAY_OF_WEEK,MKT_CARRIER_AIRLINE_ID,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_NAME,CRS_DEP_TIME,DEP_TIME,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,ARR_TIME,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,CANCELLED,CANCELLATION_CODE,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,lagged_delay_flag,prev_real_delay,FL_DATE,origin_flights_day,origin_bucket,dest_flights_day,destination_bucket,distance_bucket,airline_bucket,HOUR
0,2024,1,1,1,1,19393,10140,1014005,"Albuquerque, NM",10423,1042302,"Austin, TX",700,707.0,7.0,0.0,0.0,950.0,15.0,1.0,1.0,0.0,NaN,95.0,103.0,619.0,7.0,0.0,8.0,0.0,0.0,0,0.0,2024-01-01,67,1,241,2,2,1,7
1,2024,1,1,1,1,19393,10140,1014005,"Albuquerque, NM",10423,1042302,"Austin, TX",1830,1826.0,0.0,0.0,-1.0,2112.0,2.0,0.0,0.0,0.0,NaN,100.0,106.0,619.0,0.0,0.0,0.0,0.0,0.0,0,0.0,2024-01-01,67,1,241,2,2,1,18
2,2024,1,1,1,1,19393,10140,1014005,"Albuquerque, NM",10800,1080003,"Burbank, CA",1420,1426.0,6.0,0.0,0.0,1516.0,0.0,0.0,-1.0,0.0,NaN,130.0,110.0,672.0,0.0,0.0,0.0,0.0,0.0,0,0.0,2024-01-01,67,1,90,1,3,1,14
3,2024,1,1,1,1,19393,10140,1014005,"Albuquerque, NM",10821,1082106,"Baltimore, MD",1500,1514.0,14.0,0.0,0.0,2050.0,15.0,1.0,1.0,0.0,NaN,215.0,216.0,1670.0,14.0,0.0,1.0,0.0,0.0,0,0.0,2024-01-01,67,1,265,2,4,1,15
4,2024,1,1,1,1,19393,10140,1014005,"Albuquerque, NM",11259,1125904,"Dallas, TX",530,527.0,0.0,0.0,-1.0,805.0,0.0,0.0,-1.0,0.0,NaN,105.0,98.0,580.0,0.0,0.0,0.0,0.0,0.0,0,0.0,2024-01-01,67,1,214,2,2,1,5


# KMeans Clustering

In [114]:
cluster_features = [
    "airline_bucket", 
    "origin_bucket", 
    "destination_bucket",
    "lagged_delay_flag", 
    "prev_real_delay"
]

In [115]:
""" One Hot Encode categorical features and standarize continuous"""

# cat_features = ["MKT_CARRIER_AIRLINE_ID", "ORIGIN_AIRPORT_ID", "DEST_AIRPORT_ID", "region"]
num_features = ['airline_bucket', 'origin_bucket', 'destination_bucket', 'lagged_delay_flag', 'prev_real_delay']

preprocess = ColumnTransformer(
    transformers=[
        # ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features),
        ("num", StandardScaler(), num_features)
    ]
)

In [116]:
k = 4  # Start with this but should do elbow method??
kmeans = Pipeline([
    ("prep", preprocess),
    ("cluster", KMeans(n_clusters=k, random_state=42)) # Should I set the n_init value??
])

kmeans.fit(df_completed[cluster_features])
df_completed["cluster"] = kmeans.predict(df_completed[cluster_features])

/var/folders/h1/w_jtl0654ns96y8ngd075fj40000gn/T/ipykernel_7650/4119505359.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_completed["cluster"] = kmeans.predict(df_completed[cluster_features])


In [122]:
df_completed.to_csv("data_with_clusters.csv", index=False)

In [117]:
df_completed.head()

,YEAR,QUARTER,MONTH,DAY,DAY_OF_WEEK,MKT_CARRIER_AIRLINE_ID,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_NAME,CRS_DEP_TIME,DEP_TIME,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,ARR_TIME,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,CANCELLED,CANCELLATION_CODE,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,lagged_delay_flag,prev_real_delay,FL_DATE,origin_flights_day,origin_bucket,dest_flights_day,destination_bucket,distance_bucket,airline_bucket,HOUR,cluster
0,2024,1,1,1,1,19393,10140,1014005,"Albuquerque, NM",10423,1042302,"Austin, TX",700,707.0,7.0,0.0,0.0,950.0,15.0,1.0,1.0,0.0,NaN,95.0,103.0,619.0,7.0,0.0,8.0,0.0,0.0,0,0.0,2024-01-01,67,1,241,2,2,1,7,0
1,2024,1,1,1,1,19393,10140,1014005,"Albuquerque, NM",10423,1042302,"Austin, TX",1830,1826.0,0.0,0.0,-1.0,2112.0,2.0,0.0,0.0,0.0,NaN,100.0,106.0,619.0,0.0,0.0,0.0,0.0,0.0,0,0.0,2024-01-01,67,1,241,2,2,1,18,0
2,2024,1,1,1,1,19393,10140,1014005,"Albuquerque, NM",10800,1080003,"Burbank, CA",1420,1426.0,6.0,0.0,0.0,1516.0,0.0,0.0,-1.0,0.0,NaN,130.0,110.0,672.0,0.0,0.0,0.0,0.0,0.0,0,0.0,2024-01-01,67,1,90,1,3,1,14,1
3,2024,1,1,1,1,19393,10140,1014005,"Albuquerque, NM",10821,1082106,"Baltimore, MD",1500,1514.0,14.0,0.0,0.0,2050.0,15.0,1.0,1.0,0.0,NaN,215.0,216.0,1670.0,14.0,0.0,1.0,0.0,0.0,0,0.0,2024-01-01,67,1,265,2,4,1,15,0
4,2024,1,1,1,1,19393,10140,1014005,"Albuquerque, NM",11259,1125904,"Dallas, TX",530,527.0,0.0,0.0,-1.0,805.0,0.0,0.0,-1.0,0.0,NaN,105.0,98.0,580.0,0.0,0.0,0.0,0.0,0.0,0,0.0,2024-01-01,67,1,214,2,2,1,5,0


In [118]:
df_completed["cluster"].value_counts()

cluster
0    3087770
1    3032718
2    1270339
3      53253
Name: count, dtype: int64

In [119]:
df_completed.groupby("cluster")["DEP_DELAY_NEW"].mean()

cluster
0    15.887976
1    15.284026
2    16.836080
3    24.292153
Name: DEP_DELAY_NEW, dtype: float64

In [ ]:
# for col in ["MONTH", "HOUR"]:
#     groups = [df_completed[df_completed["cluster"] == c][col] for c in df_completed["cluster"].unique()]
#     stat, p = f_oneway(*groups)
#     # N = total sample size, k = number of groups
#     N = sum(len(g) for g in groups)
#     k = len(groups)
#     eta_sq = (stat * (k - 1)) / (stat * (k - 1) + (N - k))
#     print(col, "p:", p, "eta^2:", eta_sq)

MONTH p: 1.5310119152492762e-178 eta^2: 0.00011083834102395975
HOUR p: 0.0 eta^2: 0.022194444239954425


In [121]:
categorical = ["airline_bucket", "origin_bucket", "destination_bucket", "lagged_delay_flag", "prev_real_delay"]

for col in categorical:
    table = pd.crosstab(df_completed["cluster"], df_completed[col])
    chi2, p, dof, ex = chi2_contingency(table)
    n = table.to_numpy().sum()
    r, c = table.shape
    cramer_v = np.sqrt(chi2 / (n * (min(r - 1, c - 1))))
    print(col, "p:", p, "Cramer's V:", cramer_v)

airline_bucket p: 0.0 Cramer's V: 0.994412255348119
origin_bucket p: 0.0 Cramer's V: 0.33728121953759704
destination_bucket p: 0.0 Cramer's V: 0.4431997363586964
lagged_delay_flag p: 0.0 Cramer's V: 0.99999054316274
prev_real_delay p: 0.0 Cramer's V: 0.10284966484064409


In [ ]:
CLUSTER_COL = "cluster"

FEATURES = ["MONTH", 'HOUR', 'airline_bucket', 'origin_bucket', 'destination_bucket', 'lagged_delay_flag', 'prev_real_delay']

def cluster_summary(df, cluster_col=CLUSTER_COL, features=FEATURES):
    """
    Returns a table where each row is a cluster and columns give:
    - cluster size
    - for each feature: mode and share of that mode
    """
    out = []

    for c, group in df.groupby(cluster_col):
        row = {cluster_col: c, "n": len(group)}
        for col in features:
            vc = group[col].value_counts(normalize=True)
            mode_val = vc.index[0]
            mode_pct = vc.iloc[0]
            row[f"{col}_mode"] = mode_val
            row[f"{col}_mode_pct"] = mode_pct
        out.append(row)

    summary_df = pd.DataFrame(out).sort_values("n", ascending=False)
    return summary_df

summary = cluster_summary(df_completed)
summary


,cluster,n,MONTH_mode,MONTH_mode_pct,HOUR_mode,HOUR_mode_pct,airline_bucket_mode,airline_bucket_mode_pct,origin_bucket_mode,origin_bucket_mode_pct,destination_bucket_mode,destination_bucket_mode_pct
1,1,1673852,7,0.103390,6,0.228630,1,0.999591,1,0.473937,4,0.488011
4,4,1548693,12,0.187311,8,0.074799,1,1.000000,4,0.517686,1,0.498845
0,0,1506354,1,0.184795,8,0.074431,1,1.000000,4,0.482776,1,0.484725
2,2,1427662,6,0.105589,17,0.146360,1,1.000000,1,0.480988,4,0.533956
3,3,1287519,7,0.092633,7,0.071591,0,1.000000,3,0.305378,3,0.305289


In [ ]:
def plot_cluster_feature(df, feature, cluster_col=CLUSTER_COL):
    ct = (
        df.groupby(cluster_col)[feature]
          .value_counts(normalize=True)
          .rename("pct")
          .reset_index()
    )

    clusters = sorted(df[cluster_col].unique())
    categories = sorted(ct[feature].unique())

    for c in clusters:
        sub = ct[ct[cluster_col] == c].set_index(feature).reindex(categories).fillna(0)
        plt.figure()
        sub["pct"].plot(kind="bar")
        plt.title(f"{feature} distribution in cluster {c}")
        plt.ylabel("Proportion")
        plt.xlabel(feature)
        plt.xticks(rotation=45, ha="right")
        plt.tight_layout()
        plt.show()

plot_cluster_feature(df_completed, "airline_bucket")
plot_cluster_feature(df_completed, "origin_bucket")
plot_cluster_feature(df_completed, "destination_bucket")
plot_cluster_feature(df_completed, "MONTH")
plot_cluster_feature(df_completed, "HOUR")

# Elbow Method

In [13]:
# X = preprocess.fit_transform(df_completed) # Tried running for 6 + minutes. Takes too long to run

# inertia = []
# k_range = range(2, 6)

# for k in k_range:
#     kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
#     kmeans.fit(X)
#     inertia.append(kmeans.inertia_)

# plt.figure()
# plt.plot(k_range, inertia, marker='o')
# plt.xlabel("Number of clusters (k)")
# plt.ylabel("Inertia (Within-Cluster Sum of Squares)")
# plt.title("Elbow Method for Flight Clustering")
# plt.show()

# Silhouette

In [14]:
# for k in range(2,6):
#     km = Pipeline([
#         ("prep", preprocess),
#         ("cluster", KMeans(n_clusters=k, random_state=42))
#     ])
#     km.fit(df_completed[cluster_features])
#     cluster_labels = km.predict(df_completed[cluster_features])
#     score = silhouette_score(
#         km.named_steps["prep"].transform(df_completed[cluster_features]),
#         cluster_labels
#     )
#     print(f"k={k}, silhouette={score:.4f}")

# Other Clustering Methods

In [37]:
from kmodes.kmodes import KModes

km = KModes(n_clusters=4, init='Huang', n_init=5, verbose=1)
clusters = km.fit_predict(df_completed[["MONTH","HOUR","airline_bucket","origin_bucket","destination_bucket"]])

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 877013, cost: 19920760.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 2566630, cost: 20360887.0
Run 2, iteration: 2/100, moves: 1215998, cost: 20360887.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 1338182, cost: 20205312.0
Run 3, iteration: 2/100, moves: 778475, cost: 20205312.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 2421349, cost: 20220558.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 1969881, cost: 20091306.0
Run 5, iteration: 2/100, moves: 131511, cost: 20091306.0
Best run was number 1


In [39]:
df_completed["cluster"] = clusters

/var/folders/h1/w_jtl0654ns96y8ngd075fj40000gn/T/ipykernel_80842/2300214455.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_completed["cluster"] = clusters


In [40]:
df_completed["cluster"].value_counts()

cluster
0    3119849
3    1553932
2    1385949
1    1384350
Name: count, dtype: int64

In [41]:
df_completed.groupby("cluster")["DEP_DELAY_NEW"].mean()

cluster
0    15.822010
1    14.245656
2    16.785241
3    16.567630
Name: DEP_DELAY_NEW, dtype: float64

In [47]:
for col in ["MONTH", "HOUR"]:
    groups = [df_completed[df_completed["cluster"] == c][col] for c in df_completed["cluster"].unique()]
    stat, p = f_oneway(*groups)
    # N = total sample size, k = number of groups
    N = sum(len(g) for g in groups)
    k = len(groups)
    eta_sq = (stat * (k - 1)) / (stat * (k - 1) + (N - k))
    print(col, "p:", p, "eta^2:", eta_sq)

MONTH p: 0.0 eta^2: 0.01342031006270565
HOUR p: 0.0 eta^2: 0.0408986325478423


In [48]:
categorical = ["airline_bucket", "origin_bucket", "destination_bucket"]

for col in categorical:
    table = pd.crosstab(df_completed["cluster"], df_completed[col])
    chi2, p, dof, ex = chi2_contingency(table)
    n = table.to_numpy().sum()
    r, c = table.shape
    cramer_v = np.sqrt(chi2 / (n * (min(r - 1, c - 1))))
    print(col, "p:", p, "Cramer's V:", cramer_v)

airline_bucket p: 0.0 Cramer's V: 0.07336909235556868
origin_bucket p: 0.0 Cramer's V: 0.5081190323941394
destination_bucket p: 0.0 Cramer's V: 0.5489589718484102
